<a href="https://colab.research.google.com/github/FelixSchmidt92/question_generation/blob/master/question_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data preparation and training an answer aware question generation model. Make sure to switch to GPU runtime for training.

In [6]:
!git clone https://github.com/FelixSchmidt92/question_generation.git
%cd question_generation

Cloning into 'question_generation'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 18 (delta 4), reused 14 (delta 2), pack-reused 0
Unpacking objects: 100% (18/18), done.
/content/question_generation


In [7]:
!python build_dataset.py

File downloaded successfully!
File downloaded successfully!
100% 442/442 [02:51<00:00,  3.18it/s]
100% 35/35 [00:18<00:00,  1.49it/s]


In [8]:
%cd ..
!git clone https://github.com/OpenNMT/OpenNMT-py.git
%cd OpenNMT-py
!python setup.py install

/content
Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15547 (delta 6), reused 6 (delta 3), pack-reused 15529
Receiving objects: 100% (15547/15547), 146.48 MiB | 33.32 MiB/s, done.
Resolving deltas: 100% (11191/11191), done.
/content/OpenNMT-py
running install
running bdist_egg
running egg_info
creating OpenNMT_py.egg-info
writing OpenNMT_py.egg-info/PKG-INFO
writing dependency_links to OpenNMT_py.egg-info/dependency_links.txt
writing entry points to OpenNMT_py.egg-info/entry_points.txt
writing requirements to OpenNMT_py.egg-info/requires.txt
writing top-level names to OpenNMT_py.egg-info/top_level.txt
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/o

In [9]:
!pip install torchtext==0.4.0

     |████████████████████████████████| 61kB 2.3MB/s 
ERROR: opennmt-py 1.0.2 has requirement tqdm~=4.30.0, but you'll have tqdm 4.28.1 which is incompatible.
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [10]:
%cd /content
!mkdir preprocessed

/content


In [11]:
%cd /content/OpenNMT-py/
!python preprocess.py -train_src ../question_generation/data/squad/train/train.sentence -train_tgt ../question_generation/data/squad/train/train.question -valid_src ../question_generation/data/squad/dev/dev.sentence -valid_tgt ../question_generation/data/squad/dev/dev.question -save_data ../preprocessed/data -src_vocab_size 45000 -tgt_vocab_size 28000 -src_seq_length 100

/content/OpenNMT-py
[2020-03-08 11:28:23,713 INFO] Extracting features...
[2020-03-08 11:28:23,713 INFO]  * number of source features: 1.
[2020-03-08 11:28:23,713 INFO]  * number of target features: 0.
[2020-03-08 11:28:23,713 INFO] Building `Fields` object...
[2020-03-08 11:28:23,714 INFO] Building & saving training data...
[2020-03-08 11:28:24,033 INFO] Building shard 0.
[2020-03-08 11:28:30,509 INFO]  * saving 0th train data shard to ../preprocessed/data.train.0.pt.
[2020-03-08 11:28:36,181 INFO]  * tgt vocab size: 28004.
[2020-03-08 11:28:36,342 INFO]  * src vocab size: 45002.
[2020-03-08 11:28:36,343 INFO]  * src_feat_0 vocab size: 4.
[2020-03-08 11:28:36,648 INFO] Building & saving validation data...
[2020-03-08 11:28:36,884 INFO] Building shard 0.
[2020-03-08 11:28:37,209 INFO]  * saving 0th valid data shard to ../preprocessed/data.valid.0.pt.


In [7]:
%cd /content/preprocessed/
!mkdir glove
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove

/content/preprocessed
mkdir: cannot create directory ‘glove’: File exists
--2020-03-08 11:45:14--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-03-08 11:45:14--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-03-08 11:45:15--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [8]:
%cd /content/OpenNMT-py/
!python ./tools/embeddings_to_torch.py -emb_file_both ../preprocessed/glove/glove.6B.300d.txt \
-dict_file ../preprocessed/data.vocab.pt \
-output_file ../preprocessed/embeddings


/content/OpenNMT-py
[2020-03-08 11:53:58,897 INFO] From: ../preprocessed/data.vocab.pt
[2020-03-08 11:53:58,897 INFO] 	* source vocab: 45002 words
[2020-03-08 11:53:58,897 INFO] 	* target vocab: 28004 words
[2020-03-08 11:53:58,905 INFO] Reading encoder and decoder embeddings from ../preprocessed/glove/glove.6B.300d.txt
[2020-03-08 11:54:07,249 INFO] 	Found 400000 total vectors in file
[2020-03-08 11:54:07,249 INFO] After filtering to vectors in vocab:
[2020-03-08 11:54:07,264 INFO] 	* enc: 23747 match, 21255 missing, (52.77%)
[2020-03-08 11:54:07,272 INFO] 	* dec: 14376 match, 13628 missing, (51.34%)
[2020-03-08 11:54:07,273 INFO] 
Saving embedding as:
	* enc: ../preprocessed/embeddings.enc.pt
	* dec: ../preprocessed/embeddings.dec.pt
[2020-03-08 11:54:08,092 INFO] 
Done.


Save preprocessed data to drive for later usage.

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
%cd ../gdrive/My\ Drive
%mkdir question_generation_preprocessed
%cp -r ../../preprocessed/data.train.0.pt question_generation_preprocessed
%cp -r ../../preprocessed/data.valid.0.pt question_generation_preprocessed
%cp -r ../../preprocessed/embeddings.dec.pt question_generation_preprocessed
%cp -r ../../preprocessed/embeddings.enc.pt question_generation_preprocessed
%cp -r ../../preprocessed/data.vocab.pt question_generation_preprocessed

/content/gdrive/My Drive
mkdir: cannot create directory ‘question_generation_preprocessed’: File exists


In [10]:
%cd /content/
!mkdir save_dir
%cd OpenNMT-py/

/content
mkdir: cannot create directory ‘save_dir’: File exists
/content/OpenNMT-py


This will take a while (around 2 hours)

In [13]:
!python train.py -save_model ../save_dir --save_checkpoint_steps 1296 -valid_steps 1296 -train_steps 25920 -start_decay_steps 10500 -gpu_ranks 0 -rnn_type LSTM -encoder_type brnn -world_size 1 -enc_layers 2 -dec_layers 2 -dropout 0.3 -optim sgd -learning_rate 1 -layers 2 -bridge -global_attention general -pre_word_vecs_enc "../preprocessed/embeddings.enc.pt" -pre_word_vecs_dec "../preprocessed/embeddings.dec.pt" -rnn_size 600 -word_vec_size 300 -fix_word_vecs_enc -fix_word_vecs_dec -data ../preprocessed/data

[2020-03-08 12:02:18,847 INFO]  * src vocab size = 45002
[2020-03-08 12:02:18,847 INFO]  * src_feat_0 vocab size = 4
[2020-03-08 12:02:18,847 INFO]  * tgt vocab size = 28004
[2020-03-08 12:02:18,847 INFO] Building model...
[2020-03-08 12:02:22,894 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(45002, 300, padding_idx=1)
          (1): Embedding(4, 2, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(302, 300, num_layers=2, dropout=0.3, bidirectional=True)
    (bridge): ModuleList(
      (0): Linear(in_features=600, out_features=600, bias=True)
      (1): Linear(in_features=600, out_features=600, bias=True)
    )
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(28004, 300, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)

After training select the checkpoint with lowest perplexity and download or save on drive.
Example usage:

In [0]:
!head ../question_generation/data/squad/dev/dev.sentence > test.txt

In [17]:
!head test.txt

The￨0 Normans￨0 (￨0 Norman￨0 :￨0 Nourmands￨0 ;￨0 French￨0 :￨0 Normands￨0 ;￨0 Latin￨0 :￨0 Normanni￨0 )￨0 were￨0 the￨0 people￨0 who￨0 in￨0 the￨0 10th￨0 and￨0 11th￨0 centuries￨0 gave￨0 their￨0 name￨0 to￨0 Normandy￨0 ,￨0 a￨0 region￨0 in￨0 France￨1 .￨0
The￨0 Normans￨0 (￨0 Norman￨0 :￨0 Nourmands￨0 ;￨0 French￨0 :￨0 Normands￨0 ;￨0 Latin￨0 :￨0 Normanni￨0 )￨0 were￨0 the￨0 people￨0 who￨0 in￨0 the￨0 10th￨1 and￨1 11th￨1 centuries￨1 gave￨0 their￨0 name￨0 to￨0 Normandy￨0 ,￨0 a￨0 region￨0 in￨0 France￨0 .￨0
They￨0 were￨0 descended￨0 from￨0 Norse￨0 (￨0 "￨0 Norman￨0 "￨0 comes￨0 from￨0 "￨0 Norseman￨0 "￨0 )￨0 raiders￨0 and￨0 pirates￨0 from￨0 Denmark￨1 ,￨1 Iceland￨1 and￨1 Norway￨1 who￨0 ,￨0 under￨0 their￨0 leader￨0 Rollo￨0 ,￨0 agreed￨0 to￨0 swear￨0 fealty￨0 to￨0 King￨0 Charles￨0 III￨0 of￨0 West￨0 Francia￨0 .￨0
They￨0 were￨0 descended￨0 from￨0 Norse￨0 (￨0 "￨0 Norman￨0 "￨0 comes￨0 from￨0 "￨0 Norseman￨0 "￨0 )￨0 raiders￨0 and￨0 pirates￨0 from￨0 Denmark￨0 ,￨0 Iceland￨0 and￨0 Norway￨0 who￨0 ,￨0 under￨0 their￨0 le

In [19]:
!python translate.py -model ../save_dir_step_12960.pt -src test.txt -replace_unk -verbose

[2020-03-08 13:03:44,900 INFO] Translating shard 0.

SENT 1: ['The', 'Normans', '(', 'Norman', ':', 'Nourmands', ';', 'French', ':', 'Normands', ';', 'Latin', ':', 'Normanni', ')', 'were', 'the', 'people', 'who', 'in', 'the', '10th', 'and', '11th', 'centuries', 'gave', 'their', 'name', 'to', 'Normandy', ',', 'a', 'region', 'in', 'France', '.']
PRED 1: Where is the region of name ?
PRED SCORE: -6.5777

SENT 2: ['The', 'Normans', '(', 'Norman', ':', 'Nourmands', ';', 'French', ':', 'Normands', ';', 'Latin', ':', 'Normanni', ')', 'were', 'the', 'people', 'who', 'in', 'the', '10th', 'and', '11th', 'centuries', 'gave', 'their', 'name', 'to', 'Normandy', ',', 'a', 'region', 'in', 'France', '.']
PRED 2: When were the people of the Slavs Empire ?
PRED SCORE: -10.3564

SENT 3: ['They', 'were', 'descended', 'from', 'Norse', '(', '"', 'Norman', '"', 'comes', 'from', '"', 'Norseman', '"', ')', 'raiders', 'and', 'pirates', 'from', 'Denmark', ',', 'Iceland', 'and', 'Norway', 'who', ',', 'under', 'th